# Map of Renewable Assets in the UK

The map idea is based on: http://ukdataexplorer.com/renewables/

There is also a plan to include production data using ROC info, ELEXON (BMRS) data, or data from ENTSOE. For example as done here: http://www.renewables-map.co.uk/

Map data is from: https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
ROC data would be from: https://www.renewablesandchp.ofgem.gov.uk/Public/ReportManager.aspx?ReportVisibility=1&ReportCategory=0 

In [1]:
# import libraries

import pandas as pd # for dataframes
import numpy as np  # for arrays

from pyproj import Proj, transform # for coordinate transformation

from bokeh.plotting import figure, output_notebook, show # for plotting map
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.palettes import Category20, inferno, plasma, viridis
from bokeh.models import WMTSTileSource
from bokeh.models import CategoricalColorMapper, Legend

output_notebook() # to plot to notebook

Loading BokehJS ...

In [2]:
# import and clean data

# data from: https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
file_name = r'renewable-energy-planning-database-december-2019.csv'
renewablesData = pd.read_csv(file_name,encoding='ISO-8859-1',skiprows=1)

# only keep sites which are operational
renewablesData = renewablesData[pd.notnull(renewablesData['Operational'])]

# shorten the data set to useful parameters
coordinates = renewablesData[['Site Name','X-coordinate','Y-coordinate','Installed Capacity (MWelec)','Technology Type','Operational']]
coordinates = coordinates[pd.notnull(coordinates['X-coordinate'])]

# convert coordinates to values from strings
coordinates['X-coordinate'] = pd.to_numeric(coordinates['X-coordinate'].str.replace(',',''),errors='coerce')
coordinates['Y-coordinate'] = pd.to_numeric(coordinates['Y-coordinate'].str.replace(',',''),errors='coerce')
coordinates['Installed Capacity (MWelec)'] = pd.to_numeric(coordinates['Installed Capacity (MWelec)'].str.replace(',',''),errors='coerce')


In [3]:
# convert OSBG coordinates (Ordance Surver British National Grid)

# epsg coordinate system codes for transformations https://epsg.io/27700
#epsg:27700 is for OSBG
#epsg:3857 is for OpenMaps
#epsg:4326 is for longitude and latitude

# transformation done using Proj, http://proj4.org/usage/resource_files.html 

inProj = Proj(init='epsg:27700')
outProj = Proj(init='epsg:3857')

coordXOS = coordinates['X-coordinate'].values
coordYOS = coordinates['Y-coordinate'].values

coordX,coordY = transform(inProj,outProj,coordXOS,coordYOS)
coordinates['X-coordinateOM'],coordinates['Y-coordinateOM'] = transform(inProj,outProj,coordXOS,coordYOS)

inProj = Proj(init='epsg:27700')
outProj = Proj(init='epsg:4326')

coordinates['Longitude'],coordinates['Latitude'] = transform(inProj,outProj,coordXOS,coordYOS)


C:\Users\Charlie\anaconda3\envs\renewables-map-env39\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Charlie\anaconda3\envs\renewables-map-env39\lib\site-packages\pyproj\crs\crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
C:\Users\Charlie\anaconda3\envs\renewables-map-env39\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<auth

In [4]:
# create map plot

# See https://wiki.openstreetmap.org/wiki/Tile_servers for various tile services
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

tile_map = 'ESRI'

plot_width  = int(900)
plot_height = int(plot_width//1.2)
tools='pan,zoom_in,zoom_out,wheel_zoom,box_zoom,reset'

p = figure(tools=tools, 
        plot_width=plot_width, 
        plot_height=plot_height,
        x_axis_label="longitude",
        y_axis_label="latitude",
        match_aspect=True,
        x_axis_type="mercator",
        y_axis_type="mercator")


p.add_tile(MAP_TILES[tile_map])

p.add_layout(Legend(), 'right') # neat trick to make the legend appear outside the plot (https://discourse.bokeh.org/t/adding-a-legend-outside-the-plot-area-is-possible-even-with-auto-grouped-indirectly-created-legends/5595)

technologyTypes = coordinates['Technology Type'].unique()

colours = viridis(len(technologyTypes))

for tech,colour in zip(technologyTypes,colours):
    
    techData = coordinates.loc[coordinates['Technology Type'] == tech]
    
    source = ColumnDataSource(data=dict(
        x=techData['X-coordinateOM'],
        y=techData['Y-coordinateOM'],
        radius=2*techData['Installed Capacity (MWelec)'].values**0.5,
        sizeMW=techData['Installed Capacity (MWelec)'].values,
        technology=techData['Technology Type'],
        longitude=techData['Longitude'].values,
        latitude=techData['Latitude'].values,
        operational=techData['Operational'],
        siteName=techData['Site Name'],
    ))
    
    p.scatter('x', 'y',
          size='radius', 
          alpha=0.9, 
          fill_color=colour, 
          hover_fill_color=colour, 
          line_color="black",
          hover_line_color="white",
          fill_alpha=0.3, 
          hover_alpha=0.5,
          line_width=1, 
          legend_label=tech,
          source=source)

p.legend.click_policy="hide"

# add custom hover tool
hover = HoverTool(tooltips=[
    ("Site name","@siteName"),
    ("Technology", "@technology"),
    ("Size MW", "@sizeMW"),
    ("Operational","@operational")
    ])

p.add_tools(hover)
  

In [5]:
# show the plot
show(p) 